# P1

In [ ]:
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA, SparsePCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle

from skimage.exposure import equalize_hist
from skimage.filters import gaussian

%matplotlib inline

warnings.filterwarnings('ignore')

Let's show some images from dataset.

In [ ]:
size = 80
shape =  2 * (size,)

X_train, y_train = shuffle(np.load('../../data/pml/x_train.npy'), np.load('../../data/pml/y_train.npy'))
X_test = np.load('../../data/pml/x_test.npy')

width, height = 2, 2

plt.figure(figsize=(8, 10))
for n, (image, name) in enumerate(zip(X_train, y_train), 1):
    if n > width * height:
        break
        
    plt.subplot(height, width, n)
    plt.title(name)
    plt.imshow(image.reshape(shape), cmap='gray')

Define simple image preparing.

In [ ]:
def prepare(img):
    img = img.reshape(shape)
    img = equalize_hist(img)
    img = gaussian(img, sigma=1)
    img - img.mean()
    
    return img.flatten()
    
def transform(X):
    height, width = X.shape
    for i in range(height):
        X[i] = prepare(X[i]) 
        
    return X

In [ ]:
X_train = transform(X_train)
X_test = transform(X_test)

In [ ]:
d=np.concatenate((X_train,X_test))
d.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(d)
X_train_transform=scaler.transform(X_train)
X_test_transform=scaler.transform(X_test)

Use pca decomposition to reduce the dimensionality.

In [ ]:
pca = PCA(n_components=50)
pca.fit(np.vstack([X_train_transform, X_test_transform]))

Show some components.

In [ ]:
width, height = 8, 8

plt.figure(figsize=(16, 20))
for n, component in enumerate(pca.components_, 1):
    if n > width * height:
        break
        
    plt.subplot(height, width, n)
    plt.imshow(component.reshape(shape), cmap='gray')

And in the end we use the nearest neighbors to classify faces.

from sklearn.model_selection import train_test_split,cross_val_score
trainX, testX, trainY, testY = train_test_split(X_train, y_train, test_size=0.4, random_state=0)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=2, p=1.5, metric='minkowski',algorithm= 'auto', leaf_size= 1, n_jobs= -1, weights= 'distance')
classifier.fit(pca.transform(X_train_transform), y_train)

k_range=['minkowski','cityblock','euclidean','l1','l2','manhattan','cosine']
def find_knn(a_train,b_train,a_test,b_test):
    k_scores = []
    for k in k_range:
        print (k)
        # 2. run KNeighborsClassifier with k neighbours
        clf = KNeighborsClassifier(metric=k,n_neighbors=2,algorithm= 'auto', leaf_size= 1, n_jobs= -1, weights= 'distance')
        # 3. obtain cross_val_score for KNeighborsClassifier with k neighbours
        scores = cross_val_score(clf, a_train, b_train, cv=10, scoring='accuracy')
        k_scores.append(scores.mean())
    return k_scores

In [ ]:
find_knn(trainX, trainY, testX, testY)

In [ ]:
prediction= classifier.predict(pca.transform(X_test_transform))

In [ ]:
with open('prediction36.csv', 'w') as out:
    print('Id,Name', file=out)
    for pair in enumerate(prediction, 1):
        print('%i,%s' % pair, file=out)